In [1]:
import pandas as pd
import requests
import json
import re
from urllib.parse import quote

In [2]:
url = 'https://api.elsevier.com/content/search/scopus'
with open('config.json', 'r') as f:
    API_KEY = json.load(f)
query = '"inverse reinforcement learning"  AND  ( "system"  OR  "e-learning"  OR  "stochastic"  OR  "smart grids"  OR  "control"  OR  "system controller"  OR  "control tuning"  OR  "optimization")'

In [3]:
class scopus_df:
    def __init__(self):
        self.columns = ['Authors', 'Title', 'Year', 'Cited By', 'Affiliations', 'Author Keywords', 'Source title']
        self.csv = pd.DataFrame(columns=self.columns)

    def get_authors(self, publication: dict) -> list:
        return [author['authname'] for author in publication['author']]

    def get_affiliations(self, publication: dict) -> list:
        return [aff['affilname'].replace(',', ';') for aff in publication['affiliation']]

    def append(self, publication: dict) -> None:
        try:
            authors = ','.join(self.get_authors(publication))
        except KeyError as e:
            return None
        title = publication['dc:title']
        year = re.findall(r'([\d]{4})', publication['prism:coverDisplayDate'])[0]
        source_title = publication['prism:publicationName']
        cites = publication['citedby-count']
        try:
            affiliations = ','.join(self.get_affiliations(publication))
        except KeyError as e:
            affiliations = ''
        try:
            author_kw = ','.join([ii.lstrip().strip() for ii in publication['authkeywords'].split('|')])
        except KeyError as e:
            author_kw = ''
        self.csv = self.csv.append(pd.DataFrame([[authors, title, year, cites, affiliations, author_kw, source_title]], columns=self.columns), ignore_index=True)

def query_to_scopus(url: str, query: str, api: str, start_item: int = 0) -> list:
    return requests.get(url,
                        headers={'Accept': 'application/json', 'X-ELS-APIKey': api},
                        params={'query': query, 'view': 'COMPLETE', 'start': start_item}).json()
def create_df_from_scopus(url: str, query: str, api: str, num_items: int) -> pd.DataFrame():
    assert num_items > 0
    start_item = 0
    publications = scopus_df()
    while start_item < num_items:
        response = query_to_scopus(url, query_parsed, api, start_item)
        try:
            batch = response['search-results']['entry']
        except KeyError as e:
            break
        for item in batch:
            publications.append(item)
            start_item += 1
    return publications.csv
    

In [4]:
query_parsed = f'TITLE-ABS-KEY({query})' # TODO: Save a history of all the querys with the number of results
api = API_KEY['api-key']
num_items = int(query_to_scopus(url, query_parsed, api)['search-results']['opensearch:totalResults'])
num_items

276

In [5]:
csv = create_df_from_scopus(url, query, api, num_items)

In [6]:
csv

,Authors,Title,Year,Cited By,Affiliations,Author Keywords,Source title
0,"Shou Z.,Di X.,Ye J.,Zhu H.,Zhang H.,Hampshire R.",Optimal passenger-seeking policies on E-hailin...,2020,0,"Tongji University,Columbia University in the C...","E-hailing,Imitation learning,Markov Decision P...",Transportation Research Part C: Emerging Techn...
1,"Hitomi K.,Matsui K.,Rivas A.,Corchado J.",Development of a dangerous driving suppression...,2020,0,"Osaka Institute of Technology,Universidad de S...","Blockchain,Inverse reinforcement learning,Safe...",Advances in Intelligent Systems and Computing
2,"Hidaka K.,Hayakawa K.,Nishi T.,Usui T.,Yamamot...",Generating pedestrian walking behavior conside...,2019,0,Toyota Central Research Development Laboratory...,"Inverse reinforcement learning,Pedestrian beha...",Transportation Research Part C: Emerging Techn...
3,"Rivera-Villicana J.,Zambetta F.,Harland J.,Ber...",Exploring apprenticeship learning for player m...,2019,0,"Deakin University,RMIT University","Anchorhead,Apprenticeship Learning,Interactive...",CHI PLAY 2019 - Extended Abstracts of the Annu...
4,"Lin X.,Adams S.,Beling P.",Multi-agent inverse reinforcement learning for...,2019,0,"University of Virginia,Origent Data Sciences",,Journal of Artificial Intelligence Research
...,...,...,...,...,...,...,...
241,"Jin Z.,Qian H.,Chen S.,Zhu M.",Survey of apprenticeship learning based on rew...,2008,1,School of Computer and Computing Science,"Apprenticeship learning,Inverse reinforcement ...",Huazhong Keji Daxue Xuebao (Ziran Kexue Ban)/J...
242,"Ramachandran D.,Amir E.",Bayesian inverse reinforcement learning,2007,198,University of Illinois at Urbana-Champaign,,IJCAI International Joint Conference on Artifi...
243,"Da Silva V.,Costa A.,Lima P.",Inverse reinforcement learning with evaluation,2006,7,"Universidade de Sao Paulo - USP,Instituto Supe...",,Proceedings - IEEE International Conference on...
244,"Abbeel P.,Ng A.",Apprenticeship learning via inverse reinforcem...,2004,842,Stanford University,,"Proceedings, Twenty-First International Confer..."
